In [19]:
from nba_api.stats.endpoints import leaguedashplayershotlocations
import nba_api
import json
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

In [6]:
years = ["2020-21"]

teams = ['NYK', 'UTA', 'DAL', 'CHI', 'HOU']
team_to_id = {'NYK': '1610612752', 'UTA': '1610612762'}
#knicks_dates = ['05/16/2021', '05/15/2021', '05/13/21', '05/11/21', '05/09/21', '05/07/21', '05/05/21', '05/03/21', '05/02/21', '04/28/21']
#knicks_outcome = ['W', 'W', 'W', 'L', 'W', 'L', 'L', 'W', 'W', 'W']
#knicks_date_to_id = {'05/16/2021': '1', '05/15/2021': '2', '05/13/21': '3', '05/11/21': '4', '05/09/21': '5', '05/07/21': '6', '05/05/21': '7', '05/03/21': '8', '05/02/21': '9', '04/28/21': '10'}


master_df = pd.DataFrame(columns = [
       'PLAYER_ID',
       'PLAYER_NAME',
       'TEAM_ID',
       'TEAM_ABBREVIATION',
       'AGE',
       'ZONE',
       'FGM',
       'FGA',
       'FG_PCT',
       'SEASON'
])

for team in teams:
    for j in range(len(knicks_dates)):
        raw_data = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(
            date_from_nullable = knicks_dates[j],
            date_to_nullable = knicks_dates[j], 
            team_id_nullable = team_to_id[team]
        )

        content = json.loads(raw_data.get_json())

        #Pulling the data from the dictionary
        resultSet = content['resultSets']

        data = resultSet['rowSet']

        data_filterable = []
        for player in data:
            ra = player[:5] + ['Restricted Area']
            for i in range(5,8):
                ra.append(player[i])

            paint = player[:5] + ['In The Paint (Non-RA)']
            for i in range(8,11):
                paint.append(player[i])

            mid = player[:5] + ['Mid-Range']
            for i in range(11,14):
                mid.append(player[i])

            lc3 = player[:5] + ['Left Corner 3']
            for i in range(14,17):
                lc3.append(player[i])

            rc3 = player[:5] + ['Right Corner 3']
            for i in range(17,20):
                rc3.append(player[i])

            above3 = player[:5] + ['Above the Break 3']
            for i in range(20,23):
                above3.append(player[i])

            back = player[:5] + ['Backcourt']
            for i in range(23,26):
                back.append(player[i])

            corner = player[:5] + ['Corner 3']
            for i in range(26,29):
                corner.append(player[i])

            data_filterable.append(ra)
            data_filterable.append(paint)
            data_filterable.append(mid)
            data_filterable.append(lc3)
            data_filterable.append(rc3)
            data_filterable.append(above3)
            data_filterable.append(back)
            data_filterable.append(corner)

        df_filterable = pd.DataFrame(data_filterable, columns = [
           'PLAYER_ID',
           'PLAYER_NAME',
           'TEAM_ID',
           'TEAM_ABBREVIATION',
           'AGE',
           'ZONE',
           'FGM',
           'FGA',
           'FG_PCT'])

        if knicks_outcome[j] == 'W':
            win = 1
        else:
            win = 0 
        
        df_filterable["game_id"] = knicks_date_to_id[knicks_dates[j]]
        df_filterable["win"] = win
        

        master_df = master_df.append(df_filterable)

    master_df.to_csv('knicks_data.csv')

range(0, 10)
range(0, 10)
range(0, 10)
range(0, 10)
range(0, 10)
range(0, 10)
range(0, 10)
range(0, 10)
range(0, 10)
range(0, 10)


In [9]:
knicks = pd.read_csv('knicks_data.csv')
knicks = knicks.drop(columns = ['Unnamed: 0', 'AGE', 'SEASON', 'TEAM_ID', 'PLAYER_ID', 'FG_PCT'])
knicks.head()

,PLAYER_NAME,TEAM_ABBREVIATION,ZONE,FGM,FGA,game_id,win
0,Alec Burks,NYK,Restricted Area,1.0,3.0,1,1.0
1,Alec Burks,NYK,In The Paint (Non-RA),0.0,0.0,1,1.0
2,Alec Burks,NYK,Mid-Range,0.0,0.0,1,1.0
3,Alec Burks,NYK,Left Corner 3,0.0,1.0,1,1.0
4,Alec Burks,NYK,Right Corner 3,1.0,1.0,1,1.0


In [35]:
team = knicks.groupby(['ZONE', 'game_id']).sum()
team.head()
#team['win'] = np.where(team['win']>=1, 1, 0)
#team['FG%'] = team['FGM'] / team['FGA']
#type(team)

FGM   FGA   win
ZONE              game_id                  
Above the Break 3 1        10.0  19.0  11.0
                  2         8.0  25.0  11.0
                  3         7.0  26.0  10.0
                  4         8.0  22.0   0.0
                  5         7.0  19.0   9.0

In [28]:
atb_team = team.get_group("Above the Break 3")
atb_team.head()

AttributeError: 'DataFrame' object has no attribute 'get_group'

In [ ]:
pearsonr(,team['win'])

Previous Code Below

In [17]:
from nba_api.stats.endpoints import leaguedashplayershotlocations
import nba_api
import json
import pandas as pd

years = ["2019-20", "2020-21"]
shot_clock_ranges = ['24-22', '22-18 Very Early', '18-15 Early', '15-7 Average', '7-4 Late', '4-0 Very Late']
outcome = ['W', 'L']
dict_shot_clock = {"24-22": "24-22", '22-18 Very Early': '22-18', '18-15 Early': '18-15', '15-7 Average': '15-7', '7-4 Late':'7-4', '4-0 Very Late': '4-0'}

for result in outcome:
    for year in years: 
        for elem in shot_clock_ranges:
            raw_data = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(
                season = year,
                shot_clock_range_nullable = elem,
                outcome_nullable = result
            )

            content = json.loads(raw_data.get_json())

            #Pulling the data from the dictionary
            resultSet = content['resultSets']

            data = resultSet['rowSet']
            df = pd.DataFrame(data, columns = [
               'PLAYER_ID',
               'PLAYER_NAME',
               'TEAM_ID',
               'TEAM_ABBREVIATION',
               'AGE',
               'FGM',
               'FGA',
               'FG_PCT',
               'FGM',
               'FGA',
               'FG_PCT',
               'FGM',
               'FGA',
               'FG_PCT',
               'FGM',
               'FGA',
               'FG_PCT',
               'FGM',
               'FGA',
               'FG_PCT',
               'FGM',
               'FGA',
               'FG_PCT',
               'FGM',
               'FGA',
               'FG_PCT',
               'FGM',
               'FGA',
               'FG_PCT'])

            df.head()

            df.to_csv(result + '_' + dict_shot_clock[elem] + '_shooting_' + year + '.csv')

In [26]:
from nba_api.stats.endpoints import leaguedashplayershotlocations
import nba_api
import json
import pandas as pd

years = ["2019-20", "2020-21"]
shot_clock_ranges = ['24-22', '22-18 Very Early', '18-15 Early', '15-7 Average', '7-4 Late', '4-0 Very Late']
outcome = ['L']

master_df = pd.DataFrame(columns = [
       'PLAYER_ID',
       'PLAYER_NAME',
       'TEAM_ID',
       'TEAM_ABBREVIATION',
       'AGE',
       'SHOT-CLOCK',
       'ZONE',
       'FGM',
       'FGA',
       'FG_PCT',
       'SEASON'
])

for result in outcome:
    for year in years:
        for elem in shot_clock_ranges:
            shot_clock = elem
            raw_data = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(
                season = year,
                shot_clock_range_nullable = shot_clock,
                outcome_nullable = result
            )

            content = json.loads(raw_data.get_json())

            #Pulling the data from the dictionary
            resultSet = content['resultSets']

            data = resultSet['rowSet']

            data_filterable = []
            for player in data:
                ra = player[:5] + [shot_clock, 'Restricted Area']
                for i in range(5,8):
                    ra.append(player[i])

                paint = player[:5] + [shot_clock, 'In The Paint (Non-RA)']
                for i in range(8,11):
                    paint.append(player[i])

                mid = player[:5] + [shot_clock, 'Mid-Range']
                for i in range(11,14):
                    mid.append(player[i])

                lc3 = player[:5] + [shot_clock, 'Left Corner 3']
                for i in range(14,17):
                    lc3.append(player[i])

                rc3 = player[:5] + [shot_clock, 'Right Corner 3']
                for i in range(17,20):
                    rc3.append(player[i])

                above3 = player[:5] + [shot_clock, 'Above the Break 3']
                for i in range(20,23):
                    above3.append(player[i])

                back = player[:5] + [shot_clock, 'Backcourt']
                for i in range(23,26):
                    back.append(player[i])

                corner = player[:5] + [shot_clock, 'Corner 3']
                for i in range(26,29):
                    corner.append(player[i])

                data_filterable.append(ra)
                data_filterable.append(paint)
                data_filterable.append(mid)
                data_filterable.append(lc3)
                data_filterable.append(rc3)
                data_filterable.append(above3)
                data_filterable.append(back)
                data_filterable.append(corner)

            df_filterable = pd.DataFrame(data_filterable, columns = [
           'PLAYER_ID',
           'PLAYER_NAME',
           'TEAM_ID',
           'TEAM_ABBREVIATION',
           'AGE',
           'SHOT-CLOCK',
           'ZONE',
           'FGM',
           'FGA',
           'FG_PCT'])

            season = [year] * len(df_filterable)
            df_filterable["SEASON"] = season

            master_df = master_df.append(df_filterable)
    
master_df.to_csv('L_data.csv')

In [ ]:
from nba_api.stats.endpoints import leaguedashplayershotlocations
import nba_api
import json
import pandas as pd

years = ["2020-21"]
teams = ['NYK']
dates = ['05/16/2021', '05/15/2021', '05/13/21', '05/11/21', '05/09/21', '05/07/21', '05/05/21', '05/03/21', '05/02/21', '04/28/21']
outcome = ['W', 'W', 'W', 'L', 'W', 'L', 'L', 'W', 'W', 'W']


date_to_id = {'05/16/2021': '1', '05/15/2021': '2', '05/13/21': '3', '05/11/21': '4', '05/09/21': '5', '05/07/21': '6', '05/05/21': '7', '05/03/21': '8', '05/02/21': '9', '04/28/21': '10'}
team_to_id = {'NYK': '1610612752'}

master_df = pd.DataFrame(columns = [
       'PLAYER_ID',
       'PLAYER_NAME',
       'TEAM_ID',
       'TEAM_ABBREVIATION',
       'AGE',
       'ZONE',
       'FGM',
       'FGA',
       'FG_PCT',
       'SEASON'
])

for team in teams:
    for i in range(len(dates)):
        raw_data = leaguedashplayershotlocations.LeagueDashPlayerShotLocations(
            date_from_nullable = dates[i],
            date_to_nullable = dates[i], 
            team_id_nullable = team_to_id[team]
        )

        content = json.loads(raw_data.get_json())

        #Pulling the data from the dictionary
        resultSet = content['resultSets']

        data = resultSet['rowSet']

        data_filterable = []
        for player in data:
            ra = player[:5] + ['Restricted Area']
            for i in range(5,8):
                ra.append(player[i])

            paint = player[:5] + ['In The Paint (Non-RA)']
            for i in range(8,11):
                paint.append(player[i])

            mid = player[:5] + ['Mid-Range']
            for i in range(11,14):
                mid.append(player[i])

            lc3 = player[:5] + ['Left Corner 3']
            for i in range(14,17):
                lc3.append(player[i])

            rc3 = player[:5] + ['Right Corner 3']
            for i in range(17,20):
                rc3.append(player[i])

            above3 = player[:5] + ['Above the Break 3']
            for i in range(20,23):
                above3.append(player[i])

            back = player[:5] + ['Backcourt']
            for i in range(23,26):
                back.append(player[i])

            corner = player[:5] + ['Corner 3']
            for i in range(26,29):
                corner.append(player[i])

            data_filterable.append(ra)
            data_filterable.append(paint)
            data_filterable.append(mid)
            data_filterable.append(lc3)
            data_filterable.append(rc3)
            data_filterable.append(above3)
            data_filterable.append(back)
            data_filterable.append(corner)

        df_filterable = pd.DataFrame(data_filterable, columns = [
           'PLAYER_ID',
           'PLAYER_NAME',
           'TEAM_ID',
           'TEAM_ABBREVIATION',
           'AGE',
           'ZONE',
           'FGM',
           'FGA',
           'FG_PCT'])

        if outcome[i] == 'W':
            win = 1
        else:
            win = 0 
        
        df_filterable["game_id"] = date_to_id[dates[i]]
        df_filterable["win"] = win
        

        master_df = master_df.append(df_filterable)

    master_df.to_csv('knicks_data.csv')

In [15]:
import pandas as pd
data = pd.read_csv('complete_data_formatted.csv')
data = data.drop(columns = ['Unnamed: 0'])
data.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,SHOT-CLOCK,ZONE,FGM,FGA,FG_PCT,SEASON
0,203932,Aaron Gordon,1610612753,ORL,24.0,24-22,Restricted Area,23.0,37.0,0.622,2019-20
1,203932,Aaron Gordon,1610612753,ORL,24.0,24-22,In The Paint (Non-RA),1.0,1.0,1.000,2019-20
2,203932,Aaron Gordon,1610612753,ORL,24.0,24-22,Mid-Range,0.0,1.0,0.000,2019-20
3,203932,Aaron Gordon,1610612753,ORL,24.0,24-22,Left Corner 3,0.0,0.0,0.000,2019-20
4,203932,Aaron Gordon,1610612753,ORL,24.0,24-22,Right Corner 3,0.0,0.0,0.000,2019-20


In [16]:
knicks = data[data['TEAM_ABBREVIATION']=='NYK']
knicks_2021 = knicks[knicks['SEASON'] == '2020-21']
sc = knicks_2021[knicks_2021['SHOT-CLOCK']=='15-7 Average']
zone_sc = sc[sc['ZONE']=='Restricted Area']
zone_sc = zone_sc.sort_values(by=['FG_PCT'], ascending=False)
min_zone_sc = zone_sc[zone_sc['FGA'] >= 25]
min_zone_sc.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,SHOT-CLOCK,ZONE,FGM,FGA,FG_PCT,SEASON
42112,201959,Taj Gibson,1610612752,NYK,35.0,15-7 Average,Restricted Area,56.0,73.0,0.767,2020-21
41424,203457,Nerlens Noel,1610612752,NYK,27.0,15-7 Average,Restricted Area,56.0,78.0,0.718,2020-21
41296,1629011,Mitchell Robinson,1610612752,NYK,23.0,15-7 Average,Restricted Area,53.0,78.0,0.679,2020-21
41528,1630167,Obi Toppin,1610612752,NYK,23.0,15-7 Average,Restricted Area,27.0,41.0,0.659,2020-21
40536,203944,Julius Randle,1610612752,NYK,26.0,15-7 Average,Restricted Area,75.0,129.0,0.581,2020-21
